In [59]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import re

In [3]:
class User:
    def __init__(self, tags) -> None:
        self.tags = tags

In [85]:
class TikTokScraper:
    def __init__(self) -> None:
        self.driver = None

    def create_and_run_scraper(self, options):
        self.driver = webdriver.Chrome(options=options)
        self.driver.get("https://www.tiktok.com/foryou")
        self.click_first_video()
    
    def quit_driver(self):
        self.driver.quit()

    def click_first_video(self):
        WebDriverWait(driver=self.driver, timeout=60).until(
            EC.element_to_be_clickable(
                (
                    By.XPATH,
                    "//*[@id=\"loginContainer\"]/div/div/div/div[2]"
                )
            )
        )
        self.driver.execute_script("document.evaluate('/html/body/div[5]', document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.style.display='none';")
        self.driver.execute_script("console.log('test');")
        WebDriverWait(driver=self.driver, timeout=60).until(
            EC.element_to_be_clickable(
                (
                    By.XPATH,
                    '//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/div/div[2]/div[1]',
                )
            )
        ).click()
        WebDriverWait(self.driver, 60).until(
            EC.element_to_be_clickable(
                (
                    By.XPATH,
                    "//*[@id=\"tiktok-verify-ele\"]/div/div[4]/div/a[1]"
                )
            )
        )
        self.driver.execute_script("document.getElementById('tiktok-verify-ele').style.display='none';")
        # WebDriverWait(self.driver, 60).until(
        #     EC.element_to_be_clickable(
        #         (
        #             By.XPATH,
        #             '//*[@id="app"]/div[2]/div[3]/div[2]/div[2]/div[2]/div[1]/div[1]/button[1]',
        #         )
        #     )
        # )

In [76]:
options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
options.add_argument("--disable-infobars")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
# options.add_argument("--headless=new")

In [62]:
user = User(tags=["cats", "dogs", "animals"])

In [86]:
scraper = TikTokScraper()
scraper.create_and_run_scraper(options=options)

In [83]:
scraper.driver.execute_script("document.getElementById('tiktok-verify-ele').style.display='none';")

In [84]:
scraper.quit_driver()